# EDA MIMIC-III
<img src="files/mimic-db.png" width=40%>

DB reference: https://mit-lcp.github.io/mimic-schema-spy/tables/procedures_icd.html

## 1. Load files

In [11]:
import re
import os
import ast
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SQLContext
from pyspark.sql.types import *

### 1.1 Start Spark Session

In [5]:
from pyspark.sql import SparkSession
app_name = "ClickThrough"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

### 1.2 Read all files and convert into dataframe

In [181]:
filename_LIST = os.listdir("data")
for filename_LIST_ITEM in filename_LIST:
    filename,fileformat = filename_LIST_ITEM.split('.')
    exec(filename+'_DF = sqlContext.read.format("'+fileformat+'").option("header", "true").load("data/'+filename+'.csv")')
    exec(filename+'_DF.createOrReplaceTempView("'+filename+'")')

Following files were loaded

In [23]:
filename_LIST

['ADMISSIONS.csv',
 'CALLOUT.csv',
 'CAREGIVERS.csv',
 'CHARTEVENTS.csv',
 'CPTEVENTS.csv',
 'DATETIMEEVENTS.csv',
 'DIAGNOSES_ICD.csv',
 'DRGCODES.csv',
 'D_CPT.csv',
 'D_ICD_DIAGNOSES.csv',
 'D_ICD_PROCEDURES.csv',
 'D_ITEMS.csv',
 'D_LABITEMS.csv',
 'ICUSTAYS.csv',
 'INPUTEVENTS_CV.csv',
 'INPUTEVENTS_MV.csv',
 'LABEVENTS.csv',
 'MICROBIOLOGYEVENTS.csv',
 'NOTEEVENTS.csv',
 'OUTPUTEVENTS.csv',
 'PATIENTS.csv',
 'PRESCRIPTIONS.csv',
 'PROCEDUREEVENTS_MV.csv',
 'PROCEDURES_ICD.csv',
 'SERVICES.csv',
 'TRANSFERS.csv']

## 2 Explore dataframes

### 2.1 Patient demographics

In [36]:
PATIENTS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,46520,23260.5,13429.311598142216,1,9999
SUBJECT_ID,46520,34425.77287188306,28330.40034288402,10,99999
GENDER,46520,None,None,F,M
DOB,46520,None,None,1800-07-02 00:00:00,2201-07-24 00:00:00
DOD,15759,None,None,2100-06-19 00:00:00,2211-06-10 00:00:00
DOD_HOSP,9974,None,None,2100-06-19 00:00:00,2209-01-08 00:00:00
DOD_SSN,13378,None,None,2100-06-19 00:00:00,2211-06-10 00:00:00
EXPIRE_FLAG,46520,0.33875752364574374,0.47329238222741504,0,1


In [182]:
spark.sql("""
select GENDER, count(distinct SUBJECT_ID) as COUNT, sum(EXPIRE_FLAG) as EXPIRE
from PATIENT group by GENDER
""").toPandas().style.hide_index()

GENDER,COUNT,EXPIRE
F,20399,7235
M,26121,8524


Female patients: 44% of 46,520 patients admitted between June 2001 - October 2012 were female

In [78]:
ADMISSIONS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,58976,29488.5,17025.049074819242,1,9999
SUBJECT_ID,58976,33755.5832881172,28092.726225170416,10,99999
HADM_ID,58976,149970.8095835594,28883.095213439887,100001,199999
ADMITTIME,58976,None,None,2100-06-07 19:59:00,2210-08-17 17:13:00
DISCHTIME,58976,None,None,2100-06-09 17:09:00,2210-08-24 19:43:00
DEATHTIME,5854,None,None,2100-06-19 08:15:00,2208-02-05 11:45:00
ADMISSION_TYPE,58976,None,None,ELECTIVE,URGENT
ADMISSION_LOCATION,58976,None,None,** INFO NOT AVAILABLE **,TRSF WITHIN THIS FACILITY
DISCHARGE_LOCATION,58976,None,None,DEAD/EXPIRED,SNF-MEDICAID ONLY CERTIF


In [183]:
pd.set_option('display.max_columns', None)
spark.sql("""
select NUM_ADMISSION, count(*) as COUNT, round(count(*)/465.20,1) as PERC_COUNT 
from (select count(*) as NUM_ADMISSION from ADMISSIONS group by SUBJECT_ID) group by NUM_ADMISSION order by NUM_ADMISSION
""").toPandas().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
NUM_ADMISSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,31,34,42
COUNT,38983,5160,1342,508,246,113,51,31,26,14,13,8,5,4,1,2,3,1,1,1,1,1,1,1,2,1
PERC_COUNT,83.8,11.1,2.9,1.1,0.5,0.2,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- There are 58,976 admissions between June 2001 - October 2012 with 46,520 patients
- 84% patients only had 1 admission with an avg 1.26 admissions in the 10 year period at this hospital
- 5854 patient deaths occured during stay (12.6%)

In [114]:
spark.sql("""
select DURATION_OF_STAY, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT from (
select DATEDIFF(DISCHTIME,ADMITTIME) as DURATION_OF_STAY from ADMISSIONS) GROUP BY DURATION_OF_STAY order by DURATION_OF_STAY
""").toPandas().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161
DURATION_OF_STAY,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,116,117,118,119,120,121,123,124,125,126,127,129,130,131,132,133,134,135,136,137,138,139,140,141,142,144,145,146,148,149,151,154,156,161,163,164,165,167,169,171,172,174,182,192,202,207,295
COUNT,1096,2306,5514,5083,6277,5163,4422,3868,3342,2686,2245,1867,1560,1358,1218,1062,896,811,696,610,558,498,461,399,389,330,287,285,252,250,209,179,150,137,155,152,135,108,104,109,73,97,107,68,66,60,57,56,44,57,37,46,47,45,33,30,42,37,30,27,22,26,24,23,27,19,22,20,15,16,21,15,15,15,16,16,14,15,12,14,6,13,9,7,14,10,10,12,11,6,9,7,8,6,6,7,3,8,10,5,7,7,7,4,2,3,3,8,4,2,2,4,5,7,3,3,4,3,3,1,3,5,6,2,2,5,2,1,1,3,4,2,1,1,1,4,1,1,1,2,1,2,1,1,1,2,1,1,1,1,1,1,2,2,1,1,1,1,1,1,1,1
PERC_COUNT,1.9,3.9,9.3,8.6,10.6,8.8,7.5,6.6,5.7,4.6,3.8,3.2,2.6,2.3,2.1,1.8,1.5,1.4,1.2,1.0,0.9,0.8,0.8,0.7,0.7,0.6,0.5,0.5,0.4,0.4,0.4,0.3,0.3,0.2,0.3,0.3,0.2,0.2,0.2,0.2,0.1,0.2,0.2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
spark.sql("""select round(sum(DATEDIFF(DISCHTIME,ADMITTIME))/ 58976,1) as AVG_STAY from ADMISSIONS""").show()

+--------+
|AVG_STAY|
+--------+
|    10.1|
+--------+



- Same day discharge only happens in 2% cases
- Most frequent duration of stay is 4 days (10.6%)
- Avg length of stay is 10 days

In [121]:
spark.sql("""
select INSURANCE, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT 
from ADMISSIONS GROUP BY INSURANCE order by COUNT desc
""").toPandas().T

,0,1,2,3,4
INSURANCE,Medicare,Private,Medicaid,Government,Self Pay
COUNT,28215,22582,5785,1783,611
PERC_COUNT,47.8,38.3,9.8,3.0,1.0


- 48% of admissions are covered by medicare while 38% are covered by commercial plans
- Only 13% admissions are covered by Medicaid and other Government plans
- 1% admissions had no coverage

In [124]:
spark.sql("""
select ADMISSION_TYPE, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT 
from ADMISSIONS GROUP BY ADMISSION_TYPE order by COUNT desc
""").toPandas().T

,0,1,2,3
ADMISSION_TYPE,EMERGENCY,NEWBORN,ELECTIVE,URGENT
COUNT,42071,7863,7706,1336
PERC_COUNT,71.3,13.3,13.1,2.3


- 74% of the admissions fall under unplanned medical care (Emergency and Urgent)
- Only 26% represents pre-planned or new born related admissions

In [125]:
spark.sql("""
select ADMISSION_LOCATION, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT 
from ADMISSIONS GROUP BY ADMISSION_LOCATION order by COUNT desc
""").toPandas().T

,0,1,2,3,4,5,6,7,8
ADMISSION_LOCATION,EMERGENCY ROOM ADMIT,PHYS REFERRAL/NORMAL DELI,CLINIC REFERRAL/PREMATURE,TRANSFER FROM HOSP/EXTRAM,TRANSFER FROM SKILLED NUR,** INFO NOT AVAILABLE **,HMO REFERRAL/SICK,TRANSFER FROM OTHER HEALT,TRSF WITHIN THIS FACILITY
COUNT,22754,15079,12032,8456,273,204,102,71,5
PERC_COUNT,38.6,25.6,20.4,14.3,0.5,0.3,0.2,0.1,0.0


In [129]:
CAREGIVERS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,7567,3784.0,2184.54907627791,1,999
CGID,7567,17785.33804678208,2185.76546225744,14001,21570
LABEL,7521,4953.333333333333,4208.126582380019,1390,tap
DESCRIPTION,5155,None,None,Administrator,UCO


There are 7,567 unique health care personnel in the database

### 2.2 Billing
Coded data recorded primarily for billing and administrative purposes. Includes Current Procedural Terminology (CPT) codes, Diagnosis-Related Group (DRG) codes, and International Classification of Diseases (ICD) codes

Let's look at the diagnosis ICD table. These ICD codes are generated for billing purposes at the end of the hospital stay

In [133]:
DIAGNOSES_ICD_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,651047,325524.0,187941.22469006167,1,99999
SUBJECT_ID,651047,38971.15975805126,29372.198841354115,10,99999
HADM_ID,651047,150017.74481873045,28878.068648016237,100001,199999
SEQ_NUM,651000,7.9138356374807985,6.072633414653878,1,9
ICD9_CODE,651000,25390.601889657988,28365.292859393663,0030,V9103


In [135]:
DIAGNOSES_ICD_DF.show(5)

+------+----------+-------+-------+---------+
|ROW_ID|SUBJECT_ID|HADM_ID|SEQ_NUM|ICD9_CODE|
+------+----------+-------+-------+---------+
|  1297|       109| 172335|      1|    40301|
|  1298|       109| 172335|      2|      486|
|  1299|       109| 172335|      3|    58281|
|  1300|       109| 172335|      4|     5855|
|  1301|       109| 172335|      5|     4254|
+------+----------+-------+-------+---------+
only showing top 5 rows



Create diagnosis grouping for ICD 9 codes

In [184]:
spark.sql("""
select A.*, B.SHORT_TITLE, B.LONG_TITLE, 
case when substr(A.ICD9_CODE,1,1) in ('E','V') then 'external causes of injury and supplemental classification'
when substr(A.ICD9_CODE,1,3) between 001 and 139 then 'infectious and parasitic diseases'
when substr(A.ICD9_CODE,1,3) between 140 and 239 then 'neoplasms'
when substr(A.ICD9_CODE,1,3) between 240 and 279 then 'endocrine, nutritional and metabolic diseases, and immunity disorders'
when substr(A.ICD9_CODE,1,3) between 280 and 289 then 'diseases of the blood and blood-forming organs'
when substr(A.ICD9_CODE,1,3) between 290 and 319 then 'mental disorders'
when substr(A.ICD9_CODE,1,3) between 320 and 389 then 'diseases of the nervous system and sense organs'
when substr(A.ICD9_CODE,1,3) between 390 and 459 then 'diseases of the circulatory system'
when substr(A.ICD9_CODE,1,3) between 460 and 519 then 'diseases of the respiratory system'
when substr(A.ICD9_CODE,1,3) between 520 and 579 then 'diseases of the digestive system'
when substr(A.ICD9_CODE,1,3) between 580 and 629 then 'diseases of the genitourinary system'
when substr(A.ICD9_CODE,1,3) between 630 and 679 then 'complications of pregnancy, childbirth, and the puerperium'
when substr(A.ICD9_CODE,1,3) between 680 and 709 then 'diseases of the skin and subcutaneous tissue'
when substr(A.ICD9_CODE,1,3) between 710 and 739 then 'diseases of the musculoskeletal system and connective tissue'
when substr(A.ICD9_CODE,1,3) between 740 and 759 then 'congenital anomalies'
when substr(A.ICD9_CODE,1,3) between 760 and 779 then 'certain conditions originating in the perinatal period'
when substr(A.ICD9_CODE,1,3) between 780 and 799 then 'symptoms, signs, and ill-defined conditions'
when substr(A.ICD9_CODE,1,3) between 800 and 999 then 'injury and poisoning' 
end as ICD_GROUP
from DIAGNOSES_ICD A
left join D_ICD_DIAGNOSES B
on A.ICD9_CODE = B.ICD9_CODE
""").createOrReplaceTempView('DIAGNOSES_ICD_WITH_GROUPING')

In [185]:
spark.sql("""
select ICD_GROUP, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT from (
select distinct SUBJECT_ID, ICD_GROUP from DIAGNOSES_ICD_WITH_GROUPING) GROUP BY ICD_GROUP order by COUNT desc
""").show(truncate=False)

+---------------------------------------------------------------------+-----+----------+
|ICD_GROUP                                                            |COUNT|PERC_COUNT|
+---------------------------------------------------------------------+-----+----------+
|external causes of injury and supplemental classification            |34457|58.4      |
|diseases of the circulatory system                                   |32503|55.1      |
|endocrine, nutritional and metabolic diseases, and immunity disorders|27440|46.5      |
|diseases of the respiratory system                                   |19973|33.9      |
|injury and poisoning                                                 |19318|32.8      |
|symptoms, signs, and ill-defined conditions                          |16910|28.7      |
|diseases of the genitourinary system                                 |16765|28.4      |
|diseases of the digestive system                                     |16730|28.4      |
|diseases of the bloo

Grouping of ICD 9 reference on Wiki here: https://en.wikipedia.org/wiki/List_of_ICD-9_codes
- Injury from external causes and Cardio vascular issues are the most prominant reasons for hospitalization in the data which is representative of the entire country
- 28.7% of admissions has ill-defined conditions that couldn't be coded with a specific diagnosis code, let's see how many don't have any other specific diagnosis done by the time of discharge

In [186]:
unspecified_icd_df = spark.sql("""
select SUBJECT_ID, HADM_ID, SHORT_TITLE, ICD_GROUP from DIAGNOSES_ICD_WITH_GROUPING where HADM_ID in 
(select distinct HADM_ID from DIAGNOSES_ICD_WITH_GROUPING where ICD_GROUP = 'symptoms, signs, and ill-defined conditions') 
and HADM_ID in (select distinct HADM_ID from 
(select HADM_ID, count(*) as COUNT from DIAGNOSES_ICD_WITH_GROUPING group by HADM_ID) where COUNT = 1)
order by HADM_ID
""")
unspecified_icd_df.show(truncate = False)

+----------+-------+------------------------+-------------------------------------------+
|SUBJECT_ID|HADM_ID|SHORT_TITLE             |ICD_GROUP                                  |
+----------+-------+------------------------+-------------------------------------------+
|25200     |104775 |Headache                |symptoms, signs, and ill-defined conditions|
|72040     |105193 |Headache                |symptoms, signs, and ill-defined conditions|
|20693     |126654 |Convulsions NEC         |symptoms, signs, and ill-defined conditions|
|81429     |130919 |Headache                |symptoms, signs, and ill-defined conditions|
|30583     |156666 |Headache                |symptoms, signs, and ill-defined conditions|
|19025     |157967 |Swelling in head & neck |symptoms, signs, and ill-defined conditions|
|76949     |173111 |Dysphagia NOS           |symptoms, signs, and ill-defined conditions|
|51486     |182682 |Abdmnal pain rt upr quad|symptoms, signs, and ill-defined conditions|
|43692    

Above 9 admissions are the only cases where no specific ICD9 were coded at discharge

### 2.3 Interventions
Procedures such as dialysis, imaging studies, and placement of lines

In [188]:
PROCEDURES_ICD_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,240095,120048.0,69309.60077795835,1,99999
SUBJECT_ID,240095,32851.48147191737,27771.642798004057,10,99999
HADM_ID,240095,150082.94859118265,28918.380367052585,100003,199999
SEQ_NUM,240095,4.388433744975947,3.7725485754388277,1,9
ICD9_CODE,240095,5689.421349882338,3307.8328680774034,0010,9999


### 2.4 Laboratory
Blood chemistry, hematology, urine analysis, and microbiology test results

### 2.5 Medications
Administration records of intravenous medications and medication orders

In [187]:
PRESCRIPTIONS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,4156450,2078225.5,1199863.9075241706,1,999999
SUBJECT_ID,4156450,38772.74062024083,29252.891747744863,10,99999
HADM_ID,4156450,150100.0382379194,28917.541447040403,100001,199999
ICUSTAY_ID,2708742,250004.60841527174,28775.28560850372,200001,299999
STARTDATE,4153268,None,None,2045-10-27 00:00:00,2210-08-22 00:00:00
ENDDATE,4151029,None,None,2026-11-17 00:00:00,2210-08-25 00:00:00
DRUG_TYPE,4156450,None,None,ADDITIVE,MAIN
DRUG,4156450,1.1428571428571428,0.3779644730092272,,~<IND> Rituximab
DRUG_NAME_POE,2492216,1.1428571428571428,0.3779644730092272,Cabergoline ( Dostinex),zz


### 2.6 Notes
Free text notes such as provider progress notes and hospital discharge summaries

In [203]:
spark.sql("""select TEXT from NOTEEVENTS ORDER BY length(TEXT) desc limit 1""").rdd.flatMap(lambda x:x).collect()

["SICU NN: See carevue for specifics. Events: Recieved patient in bed short of breath, increased work of breathing, tachypnic, shivering, temp 101.1. Patient electively intubated prior to cat scan due to increased work of breathing and inability to lay flat for long periods of time. Patient intubated without incident, anesthesia present, cxr obtained post intubation and read by house officer.  Placement confirmed with end tidal co2 detector. Patient transported to ct scan for ct of torso, report as per house officers unremarkable. Patient to and from ct scan without incident. Patient was oliguric for several hours and bp dropped to 70's systolic. Levophed was started, titrating to maintain map>60 and spb>100. Swan Ganz was placed to furthur evaluate fluid status and possibility of sepsis, questioning need for diuretic therapy. Left subclavian swan ganz placed without incident, attending present. CXR obtain post insertion and read by house officer. All swan data, pa pressures, pawp, and

### 2.7 Physiologic
Nurse-verified vital signs, approximately hourly (e.g., heart rate, blood pressure, respiratory rate)

### 2.8 Reports
Free text reports of electrocardiogram and imaging studies